In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from monai.losses.dice import DiceCELoss
from monai.losses.dice import DiceLoss
import torch.nn as nn
import torch

In [ ]:

def create_a_box(image_size, box_size):
    background = np.zeros((image_size,image_size))
    box = np.ones((box_size,box_size))
    background[:len(box), :len(box)] = box
    tensor = torch.Tensor(background).unsqueeze(0)
    print(tensor.size())
    return tensor


In [ ]:
'''
Create a few artificial images of size 128
'''

# box of size 1 
box_1 = create_a_box(128, 1)

# box of size 4
box_4 = create_a_box(128, 4)

# box of size 16
box_16 = create_a_box(128, 16)

# box of size 64
box_64 = create_a_box(128, 64)

# box of size 96
box_96 = create_a_box(128, 96)

# box of size 120
box_120 = create_a_box(128, 120)

# box of size 128
box_128 = create_a_box(128, 128)


'''
Create a few artificial images of size 16
'''


# box of size 1 
BOX_1 = create_a_box(16, 1)

# box of size 4
BOX_2 = create_a_box(16, 2)

# box of size 16
BOX_3 = create_a_box(16, 3)

# box of size 64
BOX_4 = create_a_box(16, 4)

# box of size 64
BOX_6 = create_a_box(16, 6)

# box of size 64
BOX_8 = create_a_box(16, 8)

# box of size 64
BOX_12 = create_a_box(16, 12)

# box of size 64
BOX_16 = create_a_box(16, 16)


In [ ]:
def random_array(size):
    return np.random.rand(size, size, 3)

array = random_array(128)

# the plt.imshow takes as input a height, width, channel(= 3) array
plt.imshow(array)

In [ ]:
def two_by_one_box(image_size):
    background = np.zeros((image_size,image_size))
    box = np.ones((1,2))
    background[:len(box), :len(box[0])] = box
    tensor = torch.Tensor(background).unsqueeze(0)
    print(tensor.size())
    return tensor
two_by_one = two_by_one_box(4)
two_by_two = create_a_box(4, 2)
bce_loss(two_by_one, two_by_two)
# number bce ok : 14
# number bce not ok : 2
200/16


In [ ]:
import matplotlib.pyplot as plt
from infer import infer_image
import numpy as np

output = infer_image('C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\checkpoint_v1.pth', 'C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\train_data\\0a1b7d6ec.jpg')
seg = (output>0.5)+0

# print(output)
plt.imshow(output, cmap="gray")


In [ ]:
!pip install efficientnet_pytorch --quiet
from efficientnet_pytorch import EfficientNet
encoder = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)


In [ ]:
print(encoder)
encoder._conv_stem.weight.size()


In [ ]:
a = np.array([[0.8, 0.7], [0.1,0.4]])
a>0.5

In [ ]:
from infer import inference
output = inference('C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\checkpoint_v3.pth', 'C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\train_data\\0a0df8299.jpg')



In [ ]:
from infer import inference
output = inference('C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\checkpoint_v3.pth', 'C:\\Users\\LouisDorge\\OneDrive - MESASIGHT GmbH\\Desktop\\toolings\\train_data\\0a0df8299.jpg')



In [ ]:
'''
Plot a box
'''

plt.imshow(BOX_6.permute(1,2,0), cmap="gray")



In [ ]:
dice_ce_loss = DiceCELoss(batch=False, lambda_dice =  1, lambda_ce = 0, reduction = "mean", include_background=False)
bce_loss = nn.BCELoss()
dice_loss = DiceLoss(batch=False, reduction = "mean", include_background=False)

def dice(input, target, epsilon = 1e-7):

    size = input.size()
    length = len(size)

    if length == 3:
        input = input.view( -1)
        target = target.view( -1)
        
        intersection = torch.sum(target * input)
        union = torch.sum(target) + torch.sum(input)

        dice_error = ( (2 * intersection + epsilon) / (union + epsilon) )

    else:
        batch, _,_,_ = size
        input = input.view(batch, -1)
        target = target.view(batch, -1)

        intersection = torch.sum(target * input, dim=1)
        union = torch.sum(target, dim=1) + torch.sum(input, dim=1)

        dice_error = ( (2 * intersection + epsilon) / (union + epsilon) )

    return torch.mean(1 - dice_error)

def dice_bce(input, target, lambda_dice, lambda_ce):

    Dloss = dice(input, target)
    # print(Dloss)

    CEloss = bce_loss(input, target)
    # print(CEloss)
    
    return lambda_dice * Dloss + lambda_ce * CEloss

DICE = dice(BOX_6, BOX_8)
# intersection = 6*6 = 36
# union = 6*6 + 8*8 = 100
# dice score = 2 * 36 / 100
# dice loss = 1 - 0.72

DICE = dice(BOX_1, BOX_2)
# dice loss = 0.6

BCE = bce_loss(BOX_1, BOX_2)
# BCE ok : 1
# BCE not ok : 3
# total BCE : 256
# return : 300 / 256 = 1.1719

DICE_BCE = dice_bce(BOX_1, BOX_2, 1, 1)
# return 1.77

# dice_bce(BOX_1.unsqueeze(0), BOX_2.unsqueeze(0), 1, 1)

bce_loss(big_tensor_1, big_tensor_2)

dice(big_tensor_1, big_tensor_2)

In [ ]:
big_tensor_1 = torch.cat((BOX_1.unsqueeze(0), BOX_2.unsqueeze(0)), dim=0)
big_tensor_2 = torch.cat((BOX_2.unsqueeze(0), BOX_2.unsqueeze(0)), dim=0)
dice_bce(big_tensor_1, big_tensor_2, 1 ,1)

In [ ]:
from unet import UNet
unet = UNet(3, 2)


In [ ]:
prediction = BOX_2
ground_truth = BOX_3

'''
CE Loss
'''

output_ce = bce_loss(prediction, ground_truth)

print(output_ce)


'''
dice
'''

output_dice = dice(prediction, ground_truth)

print(output_dice)


'''
DICE Loss
'''

output_dice = dice_loss(prediction, ground_truth)

print(output_dice)


'''
DICE CE Loss
'''

output_ce_dice = dice_ce_loss(prediction, ground_truth)

print(output_ce_dice)
